In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot

ModuleNotFoundError: No module named 'matplotlib.artist'

In [5]:
df = pd.read_csv("../data/raw/qualidade_ar.csv")

In [6]:
df.head()

,date,pm25,pm10,o3,no2,so2,co
0,2025/5/2,39,19,20,12,,1
1,2025/5/3,44,23,28,15,,2
2,2025/5/4,61,23,27,10,,1
3,2025/5/5,61,41,21,17,,2
4,2025/5/6,53,31,15,24,,5


In [7]:
px.line(df["date"])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4065 entries, 0 to 4064
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    4065 non-null   object
 1    pm25   4065 non-null   object
 2    pm10   4065 non-null   object
 3    o3     4065 non-null   object
 4    no2    4065 non-null   object
 5    so2    4065 non-null   object
 6    co     4065 non-null   object
dtypes: object(7)
memory usage: 222.4+ KB


In [9]:
df[" pm25"].isnull().sum()

0

In [10]:
df.rename(columns={" pm25":"pm25"}, inplace=True)

In [11]:
df = df.query("pm25 != ' '")

In [12]:
df.query(" pm25 == 0")

,date,pm25,pm10,o3,no2,so2,co


In [13]:
df["pm25"] = df["pm25"].astype("int32")

In [14]:
df["date"] = pd.to_datetime(df["date"])

In [15]:
df = df.sort_values(by="date", ascending=True).reset_index(drop=True)

In [16]:
df

,date,pm25,pm10,o3,no2,so2,co
0,2014-01-02,43,,24,12,,4
1,2014-01-03,32,,25,15,,5
2,2014-01-04,41,,24,9,,4
3,2014-01-05,38,,24,7,,3
4,2014-01-06,41,,41,12,,4
...,...,...,...,...,...,...,...
2752,2025-05-28,51,28,10,10,,1
2753,2025-05-29,30,19,9,13,,4
2754,2025-05-30,22,36,12,19,,7
2755,2025-05-31,60,28,19,15,,1


In [17]:
df = df[["date", "pm25"]]

In [18]:
df

,date,pm25
0,2014-01-02,43
1,2014-01-03,32
2,2014-01-04,41
3,2014-01-05,38
4,2014-01-06,41
...,...,...
2752,2025-05-28,51
2753,2025-05-29,30
2754,2025-05-30,22
2755,2025-05-31,60


In [19]:
df_novo = pd.read_csv("../data/processed/qualidade_ar_processado.csv", parse_dates=["date"])

In [20]:
df_novo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2757 entries, 0 to 2756
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    2757 non-null   datetime64[ns]
 1   pm25    2757 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 43.2 KB


In [21]:
import joblib
from prophet import Prophet

# 🔥 Carregar o modelo salvo
model = joblib.load('../app/models/prophet_model.pkl')

# 🔮 Gerar datas futuras para previsão
future = model.make_future_dataframe(periods=30)  # 30 dias, pode mudar

# ⚡ Fazer a previsão
forecast = model.predict(future)

# 🎯 Exibir resultados principais
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

             ds       yhat  yhat_lower  yhat_upper
0    2014-01-02  44.214961   16.786261   74.298150
1    2014-01-03  46.406365   17.126967   74.588915
2    2014-01-04  47.423070   20.015392   75.626039
3    2014-01-05  44.942969   16.687332   72.500035
4    2014-01-06  41.563319   14.142405   71.205290
...         ...        ...         ...         ...
2782 2025-06-27  67.636977   39.783201   97.142442
2783 2025-06-28  68.568635   40.492933   98.401560
2784 2025-06-29  65.950674   38.051777   95.535438
2785 2025-06-30  62.390693   34.253297   92.414344
2786 2025-07-01  64.028251   35.076151   92.112714

[2787 rows x 4 columns]


In [22]:
import pandas as pd
from prophet import Prophet
import joblib

# 🔥 Carregar o modelo salvo
model = joblib.load('../app/models/prophet_model.pkl')

# Previsão
future = model.make_future_dataframe(periods=30)
forecast = model.predict(future)

# Garantir tipos corretos
forecast['yhat_lower'] = pd.to_numeric(forecast['yhat_lower'], errors='coerce')
forecast['yhat_upper'] = pd.to_numeric(forecast['yhat_upper'], errors='coerce')
forecast = forecast.dropna(subset=['yhat_lower', 'yhat_upper'])

# Plotar sem erro
fig = model.plot(forecast)

NameError: name 'plt' is not defined

In [ ]:
print(forecast.dtypes)
print(forecast[['yhat_lower', 'yhat_upper']].head())


ds                            datetime64[ns]
trend                                float64
yhat_lower                           float64
yhat_upper                           float64
trend_lower                          float64
trend_upper                          float64
additive_terms                       float64
additive_terms_lower                 float64
additive_terms_upper                 float64
weekly                               float64
weekly_lower                         float64
weekly_upper                         float64
yearly                               float64
yearly_lower                         float64
yearly_upper                         float64
multiplicative_terms                 float64
multiplicative_terms_lower           float64
multiplicative_terms_upper           float64
yhat                                 float64
dtype: object
   yhat_lower  yhat_upper
0   16.784203   73.545653
1   17.928325   76.712405
2   18.660383   75.631615
3   17.456260   72.880654
4